In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import os
import torch
import argparse
import data
import util
import torch.nn as nn
import torch.optim as optim

from models import nin_store as nin
from torch.autograd import Variable
import tqdm
import time
import numpy as np

def load_pretrained(filePath):
    model = nin.Net()
    pretrained_model = torch.load(filePath)
    useState_dict = model.state_dict()
    preState_dict = pretrained_model['state_dict']
    useKeys = useState_dict.keys()
    preKeys = preState_dict.keys()
    j = 0
    for key in useKeys:
        if key.find('num_batches_tracked') == -1:
            useState_dict[key].data = preState_dict[preKeys[j]].data
            j +=1
    model.load_state_dict(useState_dict)
    model.to(device)
    return model

def BitInverse(i, key, shape):
    
    model = load_pretrained('nin.best.pth.tar')
    bin_op = util.BinOp(model)
    model.eval()
    bin_op.binarization()
    state_dict = model.state_dict()
    

    if len(shape) == 4:
        size1 = shape[1]
        size2 = shape[2]
        size3 = shape[3]
        #(state_dict[key][int(i/size1/size2/size3)][int(i/size2/size3%size1)][int(i/size3%size2)][int(i%size3)]).mul_(-1)

    if len(shape) == 1:
        state_dict[key][i].mul_(-1)

    if len(shape) == 2:
        size = state_dict[key].shape[1]
        (state_dict[key][int(i/size)][i%size]).mul_(-1)
    
    model.load_state_dict(state_dict)
    return model

In [2]:
def test(i, key, shape, memoryData = None):
    activations = []
    test_loss = 0
    correct = 0
    
    model = BitInverse(i, key, shape)
    model.eval()
    with torch.no_grad():
        for data, target in tqdm.tqdm_notebook(memoryData, leave = False):
            data, target = data.to(device), target.to(device)

            output, activation = model(data)
            #test_loss += criterion(output, target).data.item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            activations += [activation]
            
    bin_op.restore()
    acc = 100. * float(correct) / float(len(testloader.dataset))
    #print (acc,correct,len(testloader.dataset))
    return acc, activations

In [3]:
device = torch.device('cuda:1' if torch.cuda.is_available() else "cpu")

# set the seed
torch.manual_seed(1)
torch.cuda.manual_seed(1)

# prepare the data
if not os.path.isfile('./data/'+'/train_data'):
    # check the data path
    raise Exception\
            ('Please assign the correct data path with --data <DATA_PATH>')

testset = data.dataset(root='./data/', train=False)
indices = np.load("subset_CIFAR10.npy")
testloader = torch.utils.data.DataLoader(testset,
                             batch_size=512, shuffle=False, num_workers=4)

# define classes
classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# define the model
model = load_pretrained('nin.best.pth.tar')

# define the binarization operator
bin_op = util.BinOp(model)

# do the evaluation if specified
bestAcc = 86.28
memoryData = []

find_key = "conv1.weight"
print(find_key)
state_dict = model.state_dict()

for key in state_dict.keys():
    if key.find(find_key) != -1:
        total = 1
        shape = state_dict[key].shape
        use_key = key
        for t in range(len(state_dict[key].shape)):
            total *= state_dict[key].shape[t]       

for data_batches in tqdm.tqdm_notebook(testloader, leave = False):
    memoryData += [data_batches]

conv1.weight


In [5]:
indices = [0]
with tqdm.tqdm_notebook(indices) as Loader:
    for i in Loader:
        acc, activation = test(i, use_key, shape = shape, memoryData = memoryData)
        loss = bestAcc - acc
        Loader.set_description("L: %.2f"%(loss))

        torch.save(activation, find_key+str(i)+'.unhappy.pt')